#### Necessary Imports

In [4]:
import getpass
import os
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


#### Loading the model 

In [6]:
loader = TextLoader("data/data.txt")
document = loader.load()
print(document[0].page_content)

My name is Bhawna Singh, I work in Ireland's reserach center. I am situated in Dublin however I am originally from Jaipur, India. I speak Hindi and English. When I was 12, I had a dog and his name was Shimmy. 


#### Forming the Retriever 

In [7]:
text_splitter = RecursiveCharacterTextSplitter()
splits = text_splitter.split_documents(document)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
print(splits)

[Document(page_content="My name is Bhawna Singh, I work in Ireland's reserach center. I am situated in Dublin however I am originally from Jaipur, India. I speak Hindi and English. When I was 12, I had a dog and his name was Shimmy.", metadata={'source': 'data/data.txt'})]


#### Forming the retrieval chain

In [8]:
llm = ChatOpenAI(model="gpt-4o")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    memory = memory
)

#### Question Answering with RAG

In [9]:
conversation_chain.invoke("What is my name?")['answer']

'Your name is Bhawna Singh.'

In [10]:
conversation_chain.invoke("When did I have a dog?")['answer']

'Bhawna Singh had a dog named Shimmy when she was 12 years old.'

In [11]:
conversation_chain.invoke("What was my dog's name?")['answer']

"Bhawna Singh's dog's name was Shimmy."

In [12]:
conversation_chain.invoke("Where do I work?")['answer']

"Bhawna Singh works at Ireland's research center in Dublin."